In [21]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.exceptions import UndefinedMetricWarning
import warnings
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier 

warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UndefinedMetricWarning) 

dataset = np.loadtxt("./features.txt", delimiter=" ");
print(dataset.shape)

X = dataset[:, :35]
Y = dataset[:, 35]
botnetDataset = dataset[dataset[:,35]==1, :]
normalDataset = dataset[dataset[:,35]==0, :]

botnetX = botnetDataset[:,:35]
botnetY = botnetDataset[:, 35]

normalX = normalDataset[:,:35]
normalY = normalDataset[:, 35]

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledBotnetX = scaler.fit_transform(botnetX)
rescaledNormalX = scaler.fit_transform(normalX)
rescaledX = scaler.fit_transform(X)

(52580, 36)


In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

featureColumns = ["V"+str(i) for i in range(1, dataset.shape[1]+1)]
df = pd.DataFrame(dataset, columns=featureColumns)
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36
count,52580.000000,52580.000000,52580.000000,52580.000000,52580.0,52580.000000,52580.000000,52580.000000,52580.000000,52580.000000,...,5.258000e+04,5.258000e+04,5.258000e+04,5.258000e+04,52580.000000,52580.000000,5.258000e+04,52580.000000,52580.000000,52580.000000
mean,1.282103,1.002548,0.506599,4.392716,0.0,1.134119,4.899315,3.813389,2.728129,3.072898,...,1.056908e+04,9.641168e+03,3.528380e+11,4.518123e+03,20.755192,307.370730,2.961994e+03,0.002377,0.001727,0.922005
std,1.107787,0.050419,13.209787,116.384575,0.0,0.478054,117.146050,110.460058,97.378284,100.735436,...,8.914880e+05,5.622307e+05,5.770774e+13,5.939901e+05,64.693913,9051.404227,4.982749e+04,0.055285,0.039456,0.268167
min,1.000000,1.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000e+00,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.240000e+02,1.240000e+02,0.000000e+00,0.000000e+00,1.000000,1.000000,3.000000e+00,0.000000,0.000000,1.000000
50%,1.000000,1.000000,0.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.860000e+02,1.860000e+02,0.000000e+00,0.000000e+00,1.000000,1.000000,3.000000e+00,0.000000,0.000000,1.000000
75%,1.000000,1.000000,0.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.860000e+02,1.860000e+02,0.000000e+00,0.000000e+00,2.000000,3.000000,8.000000e+00,0.000000,0.000000,1.000000
max,19.000000,2.000000,1260.000000,21407.000000,0.0,34.000000,21407.000000,21407.000000,21407.000000,21407.000000,...,1.853000e+08,7.352413e+07,1.081136e+16,1.039777e+08,256.000000,347295.000000,3.384562e+06,3.000000,1.000000,1.000000


In [23]:
'''
Feature Information
Feature Set 1: Generic features
    F1 - Total Source IPs per destination IP
    F2 - Total Protocols used for communication per destination IP
    F3 - Total Bidirectional flows per destination IP
    F4 - Total Client flows per destination IP
    F5 - Total Server flows per destination IP
    F6 - Protocols used for communication represented as bit string
    
Feature Set 2: Aggregate features
    F7 : F12 - Total, Max, Min, Mean, Variance, Std of Flows per destination IP
    F13 : F18 - Total, Max, Min, Mean, Variance, Std of Packets per destination IP
    F19 : F24 - Total, Max, Min, Mean, Variance, Std of Bytes per destination IP
    F25 : F30 - Total, Max, Min, Mean, Variance, Std of SourceBytes per destination IP
    
Feature Set 3: Subnet Features
    F31 - No. of distinct IPs in dstIP/24 subnet
    F32 - Total Flows in dstIP/24 subnet
    F33 - Total Packets in dstIP/24 subnet
    
Feature Set 4: Periodic Communication Features
    F34 - Total periodic communications involved per destination IP
    F35 - Ratio of total source IPs involved in periodic communication over total source IPs involved per dst IP
'''

genericFeatures = list(range(6))
aggregateFeatures = list(range(6, 30))
subnetFeatures = list(range(30, 33))
periodicCommnFeatures = [33, 34]

genericAggregate = genericFeatures + aggregateFeatures
genericSubnet = genericFeatures + subnetFeatures
genericPeriodic = genericFeatures + periodicCommnFeatures
aggregateSubnet = aggregateFeatures + subnetFeatures
aggregatePeriodic = aggregateFeatures + periodicCommnFeatures
subnetPeriodic = subnetFeatures + periodicCommnFeatures

withoutGenericFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in genericFeatures]
withoutAggregateFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in aggregateFeatures]
withoutSubnetFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in subnetFeatures]
withoutPeriodicCommnFeatures = [columnIndex for columnIndex in range(35) if columnIndex not in periodicCommnFeatures]
allFeatures = range(35)

configuration = [genericFeatures, aggregateFeatures, subnetFeatures, periodicCommnFeatures, 
                 genericAggregate, genericSubnet, genericPeriodic, aggregateSubnet, aggregatePeriodic, subnetPeriodic,
                 withoutGenericFeatures, withoutAggregateFeatures, withoutSubnetFeatures, withoutPeriodicCommnFeatures,
                 allFeatures
                ]

configToString = { repr(genericFeatures) : "FeatureSet 1",
                   repr(aggregateFeatures) : "FeatureSet 2",
                   repr(subnetFeatures) : "FeatureSet 3",
                   repr(periodicCommnFeatures) : "FeatureSet 4",
                   repr(genericAggregate) : "FeatureSet (1,2)",
                   repr(genericSubnet) : "FeatureSet (1,3)",
                   repr(genericPeriodic) : "FeatureSet (1,4)",
                   repr(aggregateSubnet) : "FeatureSet (2,3)",
                   repr(aggregatePeriodic) : "FeatureSet (2,4)",
                   repr(subnetPeriodic) : "FeatureSet (3,4)",
                   repr(withoutGenericFeatures) : "FeatureSet (2,3,4)",
                   repr(withoutAggregateFeatures) : "FeatureSet (1,3,4)",
                   repr(withoutSubnetFeatures) : "FeatureSet (1,2,4)",
                   repr(withoutPeriodicCommnFeatures) : "FeatureSet (1,2,3)",
                   repr(allFeatures) : "All Features"
                 }

def updateColumns(columns):
    truncBotX = rescaledBotnetX[:, columns]
    truncNormalX = rescaledNormalX[:, columns]
    truncX = rescaledX[:, columns]
    return truncBotX, truncNormalX, truncX

In [24]:
def GetNextBotnetSet(rescaledX, Y):
    step = 4000
    prevStep = 0
    while step < rescaledX.shape[0]:
        yield rescaledX[prevStep:step, :], Y[prevStep:step]
        prevStep = step
        step = step + 4000
        
    return rescaledX[prevStep:, :], Y[prevStep:]

In [25]:
for config in configuration:
    truncBotX, truncNormalX, truncX = updateColumns(config)
    scores = []
    fpr = []
    fnr = []
    for botX, botY in GetNextBotnetSet(truncBotX, botnetY):
        X = np.concatenate((botX, truncNormalX))
        Y = np.concatenate((botY, normalY))
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
        model2 = RandomForestClassifier(n_estimators=100)
        model2 = model2.fit(X_train, Y_train)
        predicted = model2.predict(X_test)
        scores.append(model2.score(X_test, Y_test))
        fpr.append(metrics.precision_score(Y_test, predicted))
        fnr.append(metrics.recall_score(Y_test, predicted))
    print("Feature Config: %s, Avg Accuracy: %.3f, Avg Precision Rate: %.3f, Avg Recall Rate: %.3f" %
            (configToString[repr(config)], np.mean(scores), np.mean(fpr), np.mean(fnr)))

Feature Config: FeatureSet 1, Avg Accuracy: 0.998, Avg Precision Rate: 0.999, Avg Recall Rate: 0.997
Feature Config: FeatureSet 2, Avg Accuracy: 0.997, Avg Precision Rate: 0.999, Avg Recall Rate: 0.994
Feature Config: FeatureSet 3, Avg Accuracy: 0.962, Avg Precision Rate: 0.948, Avg Recall Rate: 0.979
Feature Config: FeatureSet 4, Avg Accuracy: 0.505, Avg Precision Rate: 0.166, Avg Recall Rate: 0.333
Feature Config: FeatureSet (1,2), Avg Accuracy: 0.998, Avg Precision Rate: 1.000, Avg Recall Rate: 0.997
Feature Config: FeatureSet (1,3), Avg Accuracy: 0.998, Avg Precision Rate: 0.999, Avg Recall Rate: 0.998
Feature Config: FeatureSet (1,4), Avg Accuracy: 0.998, Avg Precision Rate: 0.999, Avg Recall Rate: 0.997
Feature Config: FeatureSet (2,3), Avg Accuracy: 0.998, Avg Precision Rate: 0.999, Avg Recall Rate: 0.996
Feature Config: FeatureSet (2,4), Avg Accuracy: 0.997, Avg Precision Rate: 0.999, Avg Recall Rate: 0.994
Feature Config: FeatureSet (3,4), Avg Accuracy: 0.962, Avg Precision Ra

In [30]:
for config in configuration:
    truncBotX, truncNormalX, truncX = updateColumns(config)
    scores = []
    fpr = []
    fnr = []
    for botX, botY in GetNextBotnetSet(truncBotX, botnetY):
        X = np.concatenate((botX, truncNormalX))
        Y = np.concatenate((botY, normalY))
        model = RandomForestClassifier(n_estimators=100)
        scores.append( cross_val_score(model, X, Y, scoring='accuracy', cv=10).mean() )
        fpr.append( cross_val_score(model, X, Y, scoring='precision', cv=10).mean() )
        fnr.append( cross_val_score(model, X, Y, scoring='recall', cv=10).mean() )
    print("Feature Config: %s, Avg Accuracy: %.3f, Avg Precision Rate: %.3f, Avg Recall Rate: %.3f" %
            (configToString[repr(config)], np.mean(scores), np.mean(fpr), np.mean(fnr)))

Feature Config: FeatureSet 1, Avg Accuracy: 0.997, Avg Precision Rate: 0.998, Avg Recall Rate: 0.996
Feature Config: FeatureSet 2, Avg Accuracy: 0.996, Avg Precision Rate: 0.999, Avg Recall Rate: 0.993
Feature Config: FeatureSet 3, Avg Accuracy: 0.947, Avg Precision Rate: 0.946, Avg Recall Rate: 0.946
Feature Config: FeatureSet 4, Avg Accuracy: 0.506, Avg Precision Rate: 0.112, Avg Recall Rate: 0.200
Feature Config: FeatureSet (1,2), Avg Accuracy: 0.997, Avg Precision Rate: 0.999, Avg Recall Rate: 0.996
Feature Config: FeatureSet (1,3), Avg Accuracy: 0.995, Avg Precision Rate: 0.999, Avg Recall Rate: 0.991
Feature Config: FeatureSet (1,4), Avg Accuracy: 0.997, Avg Precision Rate: 0.998, Avg Recall Rate: 0.996
Feature Config: FeatureSet (2,3), Avg Accuracy: 0.995, Avg Precision Rate: 0.999, Avg Recall Rate: 0.992
Feature Config: FeatureSet (2,4), Avg Accuracy: 0.996, Avg Precision Rate: 0.999, Avg Recall Rate: 0.993
Feature Config: FeatureSet (3,4), Avg Accuracy: 0.947, Avg Precision Ra